# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities_jimmy.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,17.36,79,79,3.44,US,1717980233
1,1,georgetown,5.4112,100.3354,26.96,88,40,2.57,MY,1717980040
2,2,upata,8.0086,-62.3989,23.36,94,100,1.63,VE,1717980233
3,3,bilibino,68.0546,166.4372,12.91,39,54,1.32,RU,1717980233
4,4,port-aux-francais,-49.3500,70.2167,3.08,87,100,17.36,TF,1717980234


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat',
                                      geo=True, 
                                      tiles='OSM', 
                                      color='City', 
                                      size='Humidity', 
                                      hover_cols=['City'],
                                      frame_height=400, frame_width=600)

# Display the map
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[(city_data_df['Cloudiness'] <= 50) &
                                  (city_data_df['Max Temp'] < 25) &
                                  (city_data_df['Humidity'] <= 60) &
                                  (city_data_df['Wind Speed'] < 9)]


# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,vilyuysk,63.7553,121.6247,21.11,36,25,2.24,RU,1717980234
12,12,labrador city,52.9463,-66.9114,19.95,45,40,3.09,CA,1717980236
14,14,thompson,55.7435,-97.8558,19.09,24,20,2.06,CA,1717980236
58,58,udachny,66.4167,112.4000,21.36,32,42,4.26,RU,1717980248
68,68,aykhal,66.0000,111.5000,21.09,29,19,3.53,RU,1717980250


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
5,vilyuysk,RU,63.7553,121.6247,36,
12,labrador city,CA,52.9463,-66.9114,45,
14,thompson,CA,55.7435,-97.8558,24,
58,udachny,RU,66.4167,112.4000,32,
68,aykhal,RU,66.0000,111.5000,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vilyuysk - nearest hotel: Мария
labrador city - nearest hotel: Carol Inn
thompson - nearest hotel: Thompson Inn
udachny - nearest hotel: Вилюй
aykhal - nearest hotel: Уют
nikopol - nearest hotel: Афіна
saldanha - nearest hotel: Hoedjesbaai
brookings - nearest hotel: Quality Inn
port saint john's - nearest hotel: Outback Inn
queenstown - nearest hotel: Queens Hotel
karratha - nearest hotel: Karratha International Hotel
new norfolk - nearest hotel: Woodbridge on the Derwent
port alfred - nearest hotel: No hotel found
polatli - nearest hotel: No hotel found
dryden - nearest hotel: Holiday Inn Express
vilhena - nearest hotel: No hotel found
wapakoneta - nearest hotel: Holiday Inn Express & Suites
khovd - nearest hotel: Khovd hotel
altai - nearest hotel: Altai hotel 40000-100000T
cooma - nearest hotel: Royal Hotel
anadyr - nearest hotel: Гостевой дом
hailar - nearest hotel: 呼伦贝尔融府商务酒店
kerman - nearest hotel: حافظ
sirte - nearest hotel: فندق المدينة-سرت
taksimo - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
5,vilyuysk,RU,63.7553,121.6247,36,Мария
12,labrador city,CA,52.9463,-66.9114,45,Carol Inn
14,thompson,CA,55.7435,-97.8558,24,Thompson Inn
58,udachny,RU,66.4167,112.4000,32,Вилюй
68,aykhal,RU,66.0000,111.5000,29,Уют
80,nikopol,UA,47.5712,34.3964,52,Афіна
83,saldanha,ZA,-33.0117,17.9442,58,Hoedjesbaai
84,brookings,US,44.3114,-96.7984,35,Quality Inn
93,port saint john's,ZA,-31.6229,29.5448,60,Outback Inn
103,queenstown,ZA,-31.8976,26.8753,60,Queens Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points('Lng', 'Lat',
                              hover_cols=['City','Humidity','Hotel Name','Country'],
                              geo=True,
                              tiles='OSM',
                              color='City',
                              size = 'Humidity',
                              frame_height=400, frame_width=600)

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)